# Red Black Trees

## Introduction
Red Black Trees are self-balancing binary search trees that maintain balance through a set of properties and color-based constraints. They were invented by Rudolf Bayer in 1972 (originally called "symmetric binary B-trees") and later named "Red-Black" trees by Leo J. Guibas and Robert Sedgewick in 1978.

## Properties

A Red Black Tree follows these key properties:
1. Every node is either red or black
2. The root node is always black
3. All leaves (NIL/NULL nodes) are black
4. If a node is red, both its children must be black (no two adjacent red nodes)
5. For each node, all simple paths from the node to descendant leaves contain the same number of black nodes (this is called the "black height")

## Why Red Black Trees?

- Red Black Trees guarantee that no path from the root to a leaf is more than twice as long as any other path. This ensures that the tree remains approximately balanced, which provides:
- O(log n) time complexity for search, insert, and delete operations
- Efficient rebalancing after insertions and deletions
- Predictable performance for real-time applications

## Operations

### Insertion

When inserting a new node:
- The node is initially colored red
- Standard BST insertion is performed
- If any Red Black Tree properties are violated, rebalancing is done through:
  - Recoloring: Changing the colors of nodes
  - Rotations: Left and right rotations to restructure the tree

### Deletion

Deletion is more complex, involving:
1. Standard BST deletion
2. If the removed node or its replacement causes violations of Red Black properties
3. Rebalancing through rotations and recoloring

## Rebalancing Operations

Red Black Trees use two primary operations for rebalancing:
1. Color flips: Changing the color of nodes
2. Rotations:
  - Left rotation: Pivoting around a node to shift the structure left
  - Right rotation: Pivoting around a node to shift the structure right

## Time Complexity

- Search: O(log n)
- Insert: O(log n)
- Delete: O(log n)
- Space: O(n)   

## Advantages and Applications

- Used in many language standard libraries (C++ STL, Java TreeMap/TreeSet)
- Database indexing structures
- Computational geometry algorithms
- Linux kernel uses Red Black Trees for process scheduling
- Used in implementing certain map and set data structures

## Compared to Other Self-Balancing Trees

- Red Black Trees require less rotations than AVL trees for insertion and deletion
- While AVL trees are more strictly balanced, Red Black Trees offer better insertion and deletion performance
- Red Black Trees are essentially a type of 2-3-4 tree represented as a binary tree

## Conclusion
Red Black Trees offer a good balance between complexity of implementation and performance, making them popular in many practical applications where balanced search trees are needed.

In [1]:
class RedBlackTree:
    # Colors represented as constants
    RED = True
    BLACK = False
    
    class Node:
        def __init__(self, key):
            self.key = key
            self.left = None 
            self.right = None
            self.parent = None
            # New nodes are always colored red initially
            self.color = RedBlackTree.RED
            
    def __init__(self):
        # Create a sentinel NIL node (represents leaves)
        self.NIL = self.Node(None)
        # NIL nodes are always black (property 3)
        self.NIL.color = RedBlackTree.BLACK
        # Root starts as NIL (empty tree)
        self.root = self.NIL
        
    def left_rotate(self, x):
        # Left rotation pivots around node x to balance the tree
        # This operation preserves the BST property while changing structure
        y = x.right
        x.right = y.left
        if y.left != self.NIL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y
        
    def right_rotate(self, x):
        # Right rotation pivots around node x (mirror of left_rotate)
        # Used to rebalance the tree after insertions and deletions
        y = x.left
        x.left = y.right
        if y.right != self.NIL:
            y.right.parent = x
        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y
        
    def insert(self, key):
        # Standard BST insertion, followed by fixing RB properties
        # Create new red node with NIL children
        node = self.Node(key)
        node.left = self.NIL
        node.right = self.NIL
        
        # Find the insertion position using BST rules
        y = None
        x = self.root
        
        while x != self.NIL:
            y = x
            if node.key < x.key:
                x = x.left
            else:
                x = x.right
                
        # Connect the new node to its parent
        node.parent = y
        if y == None:
            # Tree was empty, new node becomes root
            self.root = node
        elif node.key < y.key:
            y.left = node
        else:
            y.right = node
            
        # Fix any violations of red-black properties
        self.fix_insert(node)
        
    def fix_insert(self, k):
        # Restore red-black properties after insertion
        # Main violations: red node with red parent (property 4)
        while k.parent and k.parent.color == RedBlackTree.RED:
            # Case: Parent is right child of grandparent
            if k.parent == k.parent.parent.right:
                u = k.parent.parent.left  # Uncle
                if u.color == RedBlackTree.RED:
                    # Case 1: Uncle is red - recolor and move up
                    u.color = RedBlackTree.BLACK
                    k.parent.color = RedBlackTree.BLACK
                    k.parent.parent.color = RedBlackTree.RED
                    k = k.parent.parent
                else:
                    # Case 2: Uncle is black, k is left child (triangle)
                    if k == k.parent.left:
                        # Convert to case 3 with a rotation
                        k = k.parent
                        self.right_rotate(k)
                    # Case 3: Uncle is black, k is right child (line)
                    k.parent.color = RedBlackTree.BLACK
                    k.parent.parent.color = RedBlackTree.RED
                    self.left_rotate(k.parent.parent)
            # Case: Parent is left child of grandparent (mirror image)
            else:
                u = k.parent.parent.right  # Uncle
                if u.color == RedBlackTree.RED:
                    # Case 1: Uncle is red - recolor and move up
                    u.color = RedBlackTree.BLACK
                    k.parent.color = RedBlackTree.BLACK
                    k.parent.parent.color = RedBlackTree.RED
                    k = k.parent.parent
                else:
                    # Case 2: Uncle is black, k is right child (triangle)
                    if k == k.parent.right:
                        # Convert to case 3 with a rotation
                        k = k.parent
                        self.left_rotate(k)
                    # Case 3: Uncle is black, k is left child (line)
                    k.parent.color = RedBlackTree.BLACK
                    k.parent.parent.color = RedBlackTree.RED
                    self.right_rotate(k.parent.parent)
            if k == self.root:
                break
        # Property 2: Root must be black
        self.root.color = RedBlackTree.BLACK
        
    def transplant(self, u, v):
        # Helper function for delete - replaces subtree u with subtree v
        if u.parent == None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent
        
    def minimum(self, node):
        # Find the minimum key in the subtree rooted at node
        # Used in deletion to find inorder successor
        while node.left != self.NIL:
            node = node.left
        return node
        
    def delete(self, key):
        # Find the node to delete
        z = self.search(key)
        if not z:
            return False
            
        y = z
        y_original_color = y.color
        
        # Case 1: Node has at most one child
        if z.left == self.NIL:
            x = z.right
            self.transplant(z, z.right)
        elif z.right == self.NIL:
            x = z.left
            self.transplant(z, z.left)
        # Case 2: Node has two children
        else:
            # Find successor (minimum in right subtree)
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right
            
            if y.parent == z:
                # Successor is direct child of z
                x.parent = y
            else:
                # Replace successor with its right child
                self.transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
                
            # Replace z with its successor
            self.transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color
            
        # If we removed a black node, fix violations
        if y_original_color == RedBlackTree.BLACK:
            self.fix_delete(x)
            
        return True
        
    def fix_delete(self, x):
        # Restore red-black properties after deletion
        # Main violation: Removed a black node, affecting black height (property 5)
        while x != self.root and x.color == RedBlackTree.BLACK:
            # Case: x is left child
            if x == x.parent.left:
                w = x.parent.right  # Sibling
                # Case 1: Sibling is red
                if w.color == RedBlackTree.RED:
                    w.color = RedBlackTree.BLACK
                    x.parent.color = RedBlackTree.RED
                    self.left_rotate(x.parent)
                    w = x.parent.right
                # Case 2: Sibling is black with black children
                if w.left.color == RedBlackTree.BLACK and w.right.color == RedBlackTree.BLACK:
                    w.color = RedBlackTree.RED
                    x = x.parent
                else:
                    # Case 3: Sibling is black with red left child and black right child
                    if w.right.color == RedBlackTree.BLACK:
                        w.left.color = RedBlackTree.BLACK
                        w.color = RedBlackTree.RED
                        self.right_rotate(w)
                        w = x.parent.right
                    # Case 4: Sibling is black with red right child
                    w.color = x.parent.color
                    x.parent.color = RedBlackTree.BLACK
                    w.right.color = RedBlackTree.BLACK
                    self.left_rotate(x.parent)
                    x = self.root  # Terminate the loop
            # Mirror case: x is right child
            else:
                w = x.parent.left  # Sibling
                # Case 1: Sibling is red
                if w.color == RedBlackTree.RED:
                    w.color = RedBlackTree.BLACK
                    x.parent.color = RedBlackTree.RED
                    self.right_rotate(x.parent)
                    w = x.parent.left
                # Case 2: Sibling is black with black children
                if w.right.color == RedBlackTree.BLACK and w.left.color == RedBlackTree.BLACK:
                    w.color = RedBlackTree.RED
                    x = x.parent
                else:
                    # Case 3: Sibling is black with red right child and black left child
                    if w.left.color == RedBlackTree.BLACK:
                        w.right.color = RedBlackTree.BLACK
                        w.color = RedBlackTree.RED
                        self.left_rotate(w)
                        w = x.parent.left
                    # Case 4: Sibling is black with red left child
                    w.color = x.parent.color
                    x.parent.color = RedBlackTree.BLACK
                    w.left.color = RedBlackTree.BLACK
                    self.right_rotate(x.parent)
                    x = self.root  # Terminate the loop
        # Color x black to maintain property 5 (black height)
        x.color = RedBlackTree.BLACK
        
    def search(self, key):
        # Standard BST search
        node = self.root
        while node != self.NIL and key != node.key:
            if key < node.key:
                node = node.left
            else:
                node = node.right
        return node if node != self.NIL else None

In [2]:
# Test cases for Red Black Tree implementation
def test_red_black_tree():
    # Create a new tree
    rbt = RedBlackTree()
    
    # Test 1: Insert elements and verify search
    print("Test 1: Basic insertion and search")
    values = [7, 3, 18, 10, 22, 8, 11, 26, 2, 6]
    for val in values:
        rbt.insert(val)
        assert rbt.search(val) is not None, f"Value {val} not found after insertion"
    print("✓ All values inserted and found successfully")
    
    # Test 2: Search for non-existent value
    print("\nTest 2: Search for non-existent value")
    assert rbt.search(100) is None, "Found value that shouldn't exist"
    print("✓ Non-existent value correctly returns None")
    
    # Test 3: Delete values and verify
    print("\nTest 3: Deletion")
    delete_values = [18, 11, 3]
    for val in delete_values:
        rbt.delete(val)
        assert rbt.search(val) is None, f"Value {val} still found after deletion"
    print("✓ All values deleted successfully")
    
    # Test 4: Verify remaining values
    print("\nTest 4: Verify remaining values")
    remaining_values = [val for val in values if val not in delete_values]
    for val in remaining_values:
        assert rbt.search(val) is not None, f"Value {val} should still exist"
    print("✓ All remaining values still present")
    
    # Test 5: Property verification
    print("\nTest 5: Red-Black Tree Properties")
    def verify_properties(node, black_height=-1):
        if node == rbt.NIL:
            return 1  # NIL nodes are black
        
        # Property 1: Every node is either red or black
        assert node.color in [RedBlackTree.RED, RedBlackTree.BLACK], "Invalid color"
        
        # Property 4: Red nodes should have black children
        if node.color == RedBlackTree.RED:
            assert node.left.color == RedBlackTree.BLACK, "Red node's left child must be black"
            assert node.right.color == RedBlackTree.BLACK, "Red node's right child must be black"
            
        # Property 5: Black height must be consistent
        left_height = verify_properties(node.left)
        right_height = verify_properties(node.right)
        assert left_height == right_height, "Black height mismatch"
        
        return left_height + (1 if node.color == RedBlackTree.BLACK else 0)
    
    # Property 2: Root must be black
    assert rbt.root.color == RedBlackTree.BLACK, "Root must be black"
    verify_properties(rbt.root)
    print("✓ All Red-Black Tree properties verified")

# Run the tests
print("Running Red-Black Tree Tests...\n")
test_red_black_tree()
print("\nAll tests passed successfully!")


Running Red-Black Tree Tests...

Test 1: Basic insertion and search
✓ All values inserted and found successfully

Test 2: Search for non-existent value
✓ Non-existent value correctly returns None

Test 3: Deletion
✓ All values deleted successfully

Test 4: Verify remaining values
✓ All remaining values still present

Test 5: Red-Black Tree Properties
✓ All Red-Black Tree properties verified

All tests passed successfully!
